In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import load

cred='rgb(234, 51, 86)'
cblue = 'rgb(57, 138, 242)'

# load data

In [2]:
gdp = load.load_gdp()
gdp_18 = gdp[['Country', 'GDP']].sort_values(by='GDP', ascending=False)
gdp_cap = load.load_gdp_per_capita()[['Country', 'GDP_per_capita']].sort_values(by='GDP_per_capita', ascending=False)
rankings_soccer = load.load_rankings_soccer()
rankings_cricket = load.load_rankings_cricket()
combined = pd.merge(gdp_18, rankings_soccer, on='Country', how='outer').merge(rankings_cricket, on='Country', how='outer')
combined = pd.merge(combined, gdp_cap, on='Country', how='outer')
combined.head()

,Country,GDP,Rank_soccer,Rank_cricket,GDP_per_capita
0,United States,1.948539e+13,30.0,NaN,59927.929834
1,China,1.214349e+13,73.0,NaN,8759.041578
2,Japan,4.859951e+12,29.0,NaN,38331.979398
3,Germany,3.693204e+12,11.0,NaN,44681.082581
4,India,2.652551e+12,101.0,2.0,1981.499045


# plotly plots

In [3]:
import plotly.graph_objs as go
from plotly.offline import plot

In [18]:
df = combined

soccer_vs_gdp = go.Scatter(
                            x=list(df.GDP),
                            y=list(df.Rank_soccer.rank(pct=True, ascending=False).round(2) * 100),
                            name='Soccer',
                            mode='markers',   
                            line=dict(color=cblue), #'#33CFA5'),
                            text=df.Country,
                            visible=True
                          )

cricket_vs_gdp = go.Scatter(x=list(df.GDP),
                            y=list(df.Rank_cricket.rank(pct=True, ascending=False).round(2) * 100),
                            name='Cricket',
                            visible=True,
                            mode='markers',
                            line=dict(color=cred),
                            text=df.Country
                           )

soccer_vs_gdp_cap = go.Scatter(
                            x=list(df.GDP_per_capita),
                            y=list(df.Rank_soccer.rank(pct=True, ascending=False).round(2) * 100),
                            name='Soccer',
                            mode='markers',   
                            line=dict(color=cblue), 
                            text=df.Country,
                            visible=False
                          )

cricket_vs_gdp_cap = go.Scatter(x=list(df.GDP_per_capita),
                            y=list(df.Rank_cricket.rank(pct=True, ascending=False).round(2) * 100),
                            name='Cricket',
                            visible=False,
                            mode='markers',
                            line=dict(color=cred),
                            text=df.Country
                           )


data = [soccer_vs_gdp, cricket_vs_gdp, soccer_vs_gdp_cap, cricket_vs_gdp_cap]

updatemenus = list([
    dict(active=0,
         buttons=list([   
            dict(label = 'GDP',
                 method = 'update',
                 args = [{'visible': [True, True, False, False]}]
                ),
            dict(label = 'GDP per capita',
                 method = 'update',
                 args = [{'visible': [False, False, True, True]}])
        ]),
        x=0.5, 
        y=-0.06,
        xanchor='center',
        direction='up',
        showactive=False,
        bgcolor='rgb(57, 138, 242, 40)',
        font={'color': 'black'}
    )
])

layout = dict(title='World sports rankings', 
              xaxis=dict(
                    type='log',
                    autorange=True
              ),
              yaxis=dict(
                  title="Percentile in world rankings",
              ),
              showlegend=True,
              updatemenus=updatemenus,
              template='plotly_dark'
             )

fig = dict(data=data, layout=layout)
plot(fig, filename='update_dropdown_test.html')

'file:///accounts/projects/vision/chandan/sports-analysis/update_dropdown_test.html'